In [1]:
# Install & import
!pip install pandas scikit-learn joblib
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report, mean_squared_error
import kagglehub
import polars as pl
import joblib
import shutil
import os

In [2]:
class LoadSetDataset:
    def __init__(self):
      self.path = kagglehub.dataset_download("nikhil1e9/loan-default")
      self.destination_path = '/content/kaggle/'
      self.df=None

    @staticmethod
    def load_dataset(self):
      source_path = self.path
      destination_path = self.destination_path
      shutil.copytree(source_path, destination_path, dirs_exist_ok=True)
      print("Path to dataset files:", self.path, "\n Copied Path of Dataset:", destination_path)
      return f"{destination_path}/Loan_default.csv"

    def read_dataset(self):
      self.df = pl.read_csv(self.load_dataset(self))
      print(self.df.head())
      return self.df

In [3]:
class PreprocessDataset(LoadSetDataset):
    def __init__(self, n):
      super().__init__()
      self.df = self.read_dataset()
      self.n = n

    def cleanup_imputation(self):
      # Drop rows with missing values in specified columns
      self.df = self.df.drop_nulls(subset=['Default','CreditScore','LoanAmount'])  # minimal cleanup
      print(self.df.shape)
      return self.df

    def set_range(self):
      self.df = self.df.head(self.n)
      print(self.df.shape)
      return self.df

    def __del__(self):
      del self.df

In [4]:
ldst=PreprocessDataset(1247)
ldst.cleanup_imputation()
df=ldst.set_range()

100%|██████████| 7.86M/7.86M [00:00<00:00, 108MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/nikhil1e9/loan-default/versions/2 
 Copied Path of Dataset: /content/kaggle/
shape: (5, 18)
┌────────────┬─────┬────────┬────────────┬───┬───────────────┬─────────────┬─────────────┬─────────┐
│ LoanID     ┆ Age ┆ Income ┆ LoanAmount ┆ … ┆ HasDependents ┆ LoanPurpose ┆ HasCoSigner ┆ Default │
│ ---        ┆ --- ┆ ---    ┆ ---        ┆   ┆ ---           ┆ ---         ┆ ---         ┆ ---     │
│ str        ┆ i64 ┆ i64    ┆ i64        ┆   ┆ str           ┆ str         ┆ str         ┆ i64     │
╞════════════╪═════╪════════╪════════════╪═══╪═══════════════╪═════════════╪═════════════╪═════════╡
│ I38PQUQS96 ┆ 56  ┆ 85994  ┆ 50587      ┆ … ┆ Yes           ┆ Other       ┆ Yes         ┆ 0       │
│ HPSK72WA7R ┆ 69  ┆ 50432  ┆ 124440     ┆ … ┆ No            ┆ Other       ┆ Yes         ┆ 0       │
│ C1OZ6DPJ8Y ┆ 46  ┆ 84208  ┆ 129188     ┆ … ┆ Yes           ┆ Auto        ┆ No          ┆ 1       │
│ V2KKSFM3UN ┆ 32  ┆ 31713  ┆ 44799      ┆ … 

In [5]:
class splitDataset:
  X_train, y_clf_train, y_reg_train = None, None, None
  X_test, y_clf_test, y_reg_test = None, None, None
  def __init__(self, df):
    self.df = df
    self.X = None
    self.y_clf = None
    self.y_reg = None

  def X_y_split(self):
    # One-hot encoding for categorical features
    self.X = self.df.to_pandas().drop(['Default','LoanAmount'], axis=1)
    self.X = pl.from_pandas(pd.get_dummies(self.X))
    # Define targets
    self.y_clf = self.df['Default']
    self.y_reg = self.df['LoanAmount'].fill_null(0)  # regression target
    return self.X, self.y_clf, self.y_reg

  @classmethod
  def train_test_split(cls, X, y_clf, y_reg):
    X_pd = X.to_pandas()
    y_clf_pd = y_clf.to_pandas()
    y_reg_pd = y_reg.to_pandas()
    # Train-test split
    X_train, X_test, y_clf_train, y_clf_test, y_reg_train, y_reg_test = train_test_split(
        X_pd, y_clf_pd, y_reg_pd, test_size=0.2, random_state=42
    )
    return X_train, X_test, y_clf_train, y_clf_test, y_reg_train, y_reg_test

  def process_split(self):
    X, y_clf, y_reg = self.X_y_split()
    X_train, X_test, y_clf_train, y_clf_test, y_reg_train, y_reg_test = self.train_test_split(X, y_clf, y_reg)
    return X_train, X_test, y_clf_train, y_clf_test, y_reg_train, y_reg_test

  def __del__(self):
    del self.df

In [6]:
spd=splitDataset(df)
X_train, X_test, y_clf_train, y_clf_test, y_reg_train, y_reg_test = spd.process_split()
print(X_train.shape, X_test.shape, y_clf_train.shape, y_clf_test.shape, y_reg_train.shape, y_reg_test.shape)

(997, 1277) (250, 1277) (997,) (250,) (997,) (250,)


In [9]:
from abc import ABC, abstractmethod
class Model(ABC):
  @abstractmethod
  def __init__(self, X_train, X_test, y_clf_train, y_reg_train, y_clf_test, y_reg_test):
      self.X_train = X_train
      self.X_test = X_test
      self.y_clf_train = y_clf_train
      self.y_reg_train = y_reg_train
      self.y_clf_test = y_clf_test
      self.y_reg_test = y_reg_test

  @abstractmethod
  def process_model(self):
      pass

  @abstractmethod
  def train_model(self):
      pass

  @abstractmethod
  def evaluate_model(self):
      pass

  @abstractmethod
  def run_model(self):
      pass

  @abstractmethod
  def __del__(self):
      pass

In [10]:
class regression_model(Model):
  def __init__(self, X_train, X_test, y_clf_train, y_reg_train, y_clf_test, y_reg_test):
    super().__init__(X_train, X_test, y_clf_train, y_reg_train, y_clf_test, y_reg_test)
    self.scaler = None
    self.reg = None

  def process_model(self):
    # Scaling
    self.scaler = StandardScaler()
    X_train_scaled = self.scaler.fit_transform(self.X_train)
    X_test_scaled = self.scaler.transform(self.X_test)
    return X_train_scaled, X_test_scaled

  def train_model(self, X_train_scaled, X_test_scaled):
    self.reg = RandomForestRegressor(n_estimators=100, random_state=42)
    self.reg.fit(X_train_scaled, self.y_reg_train)
    return self.reg

  def evaluate_model(self, X_test_scaled):
    print("RMSE:", mean_squared_error(self.y_reg_test, self.reg.predict(X_test_scaled)))

  def run_model(self):
    X_train_scaled, X_test_scaled = self.process_model()
    model = self.train_model(X_train_scaled, X_test_scaled)
    self.evaluate_model(X_test_scaled)
    return self.scaler, model

  def __del__(self):
    del self.scaler
    del self.reg

In [11]:
class classification_model(Model):
  def __init__(self, X_train, X_test, y_clf_train, y_reg_train, y_clf_test, y_reg_test):
    super().__init__(X_train, X_test, y_clf_train, y_reg_train, y_clf_test, y_reg_test)
    self.scaler = None
    self.clf = None

  def process_model(self):
    # Scaling
    self.scaler = StandardScaler()
    X_train_scaled = self.scaler.fit_transform(self.X_train)
    X_test_scaled = self.scaler.transform(self.X_test)
    return X_train_scaled, X_test_scaled

  def train_model(self, X_train_scaled, X_test_scaled):
    self.clf = RandomForestClassifier(n_estimators=100, random_state=42)
    self.clf.fit(X_train_scaled, self.y_clf_train)
    return self.clf

  def evaluate_model(self, X_test_scaled):
    print(classification_report(self.y_clf_test, self.clf.predict(X_test_scaled)))

  def run_model(self):
    X_train_scaled, X_test_scaled = self.process_model()
    model = self.train_model(X_train_scaled, X_test_scaled)
    self.evaluate_model(X_test_scaled)
    return model

  def __del__(self):
    del self.scaler
    del self.clf

In [12]:
regMod = regression_model(X_train, X_test, y_clf_train, y_reg_train, y_clf_test, y_reg_test)
scaler, regr = regMod.run_model()

RMSE: 5582933414.570695


In [13]:
clsMod = classification_model(X_train, X_test, y_clf_train, y_reg_train, y_clf_test, y_reg_test)
clf = clsMod.run_model()

              precision    recall  f1-score   support

           0       0.89      1.00      0.94       222
           1       0.00      0.00      0.00        28

    accuracy                           0.89       250
   macro avg       0.44      0.50      0.47       250
weighted avg       0.79      0.89      0.84       250



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [15]:
# Save everything
joblib.dump({'scaler':scaler, 'clf':clf, 'reg':regr}, 'loan_risk_model.pkl')

['loan_risk_model.pkl']

In [ ]:
# from fastapi import FastAPI, HTTPException
# from pydantic import BaseModel, Field
# import pandas as pd
# import joblib
# from typing import Dict
# import logging
# import os
# from pathlib import Path

# # Configure logging
# logging.basicConfig(level=logging.INFO)
# logger = logging.getLogger(__name__)

# app = FastAPI(title="Loan Risk Prediction API")

# # Load trained model artifacts with error handling
# MODEL_PATH = Path("loan_risk_model.pkl")
# try:
#     if not MODEL_PATH.exists():
#         raise FileNotFoundError(f"Model file {MODEL_PATH} not found")
#     model_bundle = joblib.load(MODEL_PATH)
#     scaler = model_bundle.get("scaler")
#     clf = model_bundle.get("clf")
#     reg = model_bundle.get("reg")
#     if not all([scaler, clf, reg]):
#         raise ValueError("Model bundle missing required keys: scaler, clf, reg")
# except Exception as e:
#     logger.error(f"Failed to load model: {str(e)}")
#     raise RuntimeError(f"Model loading failed: {str(e)}")

# # In-memory storage (consider replacing with a database in production)
# db: Dict[int, Dict] = {}
# applicant_id_counter: int = 0

# # Define input and response schemas
# class Applicant(BaseModel):
#     age: int = Field(..., ge=18, le=120, description="Applicant age")
#     income: float = Field(..., ge=0, description="Annual income in USD")
#     credit_score: float = Field(..., ge=300, le=850, description="Credit score")
#     loan_amount: float = Field(..., ge=0, description="Requested loan amount")
#     gender: str = Field(..., min_length=1, description="Gender")
#     purpose: str = Field(..., min_length=1, description="Loan purpose")

# class ApplicantResponse(BaseModel):
#     applicant_id: int
#     approved: bool
#     default_risk_score: float
#     input_data: Applicant

# @app.post("/applicants/", response_model=ApplicantResponse)
# async def create_applicant(data: Applicant):
#     global applicant_id_counter
#     try:
#         # Convert input to DataFrame and encode
#         df = pd.DataFrame([data.dict()])
#         df_encoded = pd.get_dummies(df)

#         # Align columns with training set
#         for col in scaler.feature_names_in_:
#             if col not in df_encoded:
#                 df_encoded[col] = 0
#         df_encoded = df_encoded[scaler.feature_names_in_]

#         # Scale and predict
#         X_scaled = scaler.transform(df_encoded)
#         prediction = bool(clf.predict(X_scaled)[0])
#         risk_score = float(reg.predict(X_scaled)[0])

#         # Store data
#         applicant_id_counter += 1
#         db[applicant_id_counter] = {
#             "input": data.dict(),
#             "prediction": prediction,
#             "risk_score": risk_score
#         }
#         logger.info(f"Created applicant ID {applicant_id_counter} with prediction: {prediction}")
#         return {
#             "applicant_id": applicant_id_counter,
#             "approved": prediction,
#             "default_risk_score": risk_score,
#             "input_data": data
#         }
#     except Exception as e:
#         logger.error(f"Error processing applicant: {str(e)}")
#         raise HTTPException(status_code=500, detail=f"Prediction failed: {str(e)}")

# @app.get("/applicants/{applicant_id}")
# async def get_applicant(applicant_id: int):
#     if applicant_id not in db:
#         logger.warning(f"Applicant ID {applicant_id} not found")
#         raise HTTPException(status_code=404, detail="Applicant not found")
#     logger.info(f"Retrieved applicant ID {applicant_id}")
#     return db[applicant_id]

# @app.put("/applicants/{applicant_id}", response_model=ApplicantResponse)
# async def update_applicant(applicant_id: int, data: Applicant):
#     if applicant_id not in db:
#         logger.warning(f"Applicant ID {applicant_id} not found")
#         raise HTTPException(status_code=404, detail="Applicant not found")
#     try:
#         # Re-predict with updated data
#         df = pd.DataFrame([data.dict()])
#         df_encoded = pd.get_dummies(df)
#         for col in scaler.feature_names_in_:
#             if col not in df_encoded:
#                 df_encoded[col] = 0
#         df_encoded = df_encoded[scaler.feature_names_in_]
#         X_scaled = scaler.transform(df_encoded)
#         prediction = bool(clf.predict(X_scaled)[0])
#         risk_score = float(reg.predict(X_scaled)[0])

#         # Update storage
#         db[applicant_id] = {
#             "input": data.dict(),
#             "prediction": prediction,
#             "risk_score": risk_score
#         }
#         logger.info(f"Updated applicant ID {applicant_id} with prediction: {prediction}")
#         return {
#             "applicant_id": applicant_id,
#             "approved": prediction,
#             "default_risk_score": risk_score,
#             "input_data": data
#         }
#     except Exception as e:
#         logger.error(f"Error updating applicant {applicant_id}: {str(e)}")
#         raise HTTPException(status_code=500, detail=f"Prediction failed: {str(e)}")

# @app.delete("/applicants/{applicant_id}")
# async def delete_applicant(applicant_id: int):
#     if applicant_id not in db:
#         logger.warning(f"Applicant ID {applicant_id} not found")
#         raise HTTPException(status_code=404, detail="Applicant not found")
#     del db[applicant_id]
#     logger.info(f"Deleted applicant ID {applicant_id}")
#     return {"deleted": True}

# # Health check endpoint
# @app.get("/health")
# async def health_check():
#     return {"status": "healthy"}

In [ ]:
# {
#   "age": 30,
#   "income": 50000.0,
#   "credit_score": 700.0,
#   "loan_amount": 10000.0,
#   "gender": "Male",
#   "purpose": "Home"
# }